In [2]:
pip install langchain langchain-community openai faiss-cpu transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install unstructured

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install chromadb

In [7]:
from langchain.document_loaders import CSVLoader

In [1]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import CSVLoader
import pandas as pd


os.environ["OPENAI_API_KEY"] = 'sk-proj-kVVV3jXuQnSTmPTQ0g8uT3BlbkFJMrJWSUTOsrztlCEa23SK'


data_loader = CSVLoader("./Health_Data.csv")
cache_dir = LocalFileStore("./.cache/")



splitter = CharacterTextSplitter(
    separator="\n", 
    chunk_size=500,
    chunk_overlap=50
)

docs = data_loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"])
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

C:\Users\SMHRD\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


client=<openai.resources.embeddings.Embeddings object at 0x000001A58B6EDB90> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000001A58B6F5B10> model='text-embedding-ada-002' deployment='text-embedding-ada-002' openai_api_version='' openai_api_base=None openai_api_type='' openai_proxy='' embedding_ctx_length=8191 openai_api_key='sk-proj-kVVV3jXuQnSTmPTQ0g8uT3BlbkFJMrJWSUTOsrztlCEa23SK' openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None
<class 'langchain_community.embeddings.openai.OpenAIEmbeddings'>


In [2]:
map_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            당신은 헬스 트레이너AI 입니다. 당신이 갖고 있는 정보를 참고하여 고객에게 알맞는 운동을 추천해주세요 : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_chain = map_prompt | ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

def map_docs(inputs):
    documents, question = inputs["documents"], inputs["question"]
    return "\n\n".join(
        map_chain.invoke({"context": doc.page_content, "question": question}).content
        for doc in documents
    )

map_results = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            주어진 문장들을 이용해 최종 답변을 작성해주세요. 만약 주어진 문장들 내에 답변을 위한 내용이 포함되어있지 않다면, 답변을 꾸며내지 말고, 모른다고 답해주세요.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

reduce_chain = {"context": map_results, "question": RunnablePassthrough()} | reduce_prompt | ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])


C:\Users\SMHRD\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [7]:
response = reduce_chain.invoke("나는 키 174에 몸무게 75kg, 30살 남자야 오늘 운동 일정을 대신 계획해줘")

print(response.content)

고객님의 정보를 확인해보니 키 174cm, 몸무게 75kg, 30세의 남성이시군요. BMI를 계산해보면 24.77로 정상 범주에 속합니다.

운동 일정을 대신 계획해드리겠습니다:

운동 종류: 인치웜 (Inchworm)
운동 부위: 복부
장비: 본인 체중만으로 수행 가능
세트 및 횟수: 3세트, 12-15회

인치웜은 복부를 중심으로 근력과 유연성을 향상시키는데 도움을 줄 수 있는 운동입니다. 꾸준히 실시하시면 몸매 개선에 도움이 될 것입니다.운동 시작 전에 충분한 스트레칭과 워밍업을 잊지말고 진행해주세요.운동 후에는 쿨다운과 스트레칭을 통해 근육의 회복을 도와주세요.운동 계획 수행 시에는 자신의 몸 상태를 듣고 무리하지 않도록 주의해주세요.


In [74]:
pip install Flask

In [75]:
pip install Flask langchain langchain-community openai faiss-cpu transformers unstructured tiktoken chromadb

Note: you may need to restart the kernel to use updated packages.


In [78]:
from flask import Flask, jsonify, request
app = Flask(__name__)

@app.route('/api/data', methods=['POST'])
def get_recommendation():
    data = request.json
    question = data.get("question")
    response = reduce_chain.invoke(question)
    return jsonify({"response": response.content})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.30.1.47:5000
Press CTRL+C to quit
127.0.0.1 - - [28/May/2024 09:51:09] "POST /api/data HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 2548, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.handle_exception(e)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SMHRD\anaconda3\Lib\site-packag

172.30.1.47 - - [28/May/2024 09:56:53] "POST /api/data HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 2548, in __call__
    return self.wsgi_app(environ, start_response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 2528, in wsgi_app
    response = self.handle_exception(e)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\SMHRD\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\